<a href="https://colab.research.google.com/github/kupieckozienice/pyPDF2/blob/main/zapytanie_RFX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `klonowanie repozytorium`

In [172]:
# @title Clone the entire repo { vertical-output: true }
repo_link = "https://github.com/kupieckozienice/pyPDF2" # @param {type:"string"}

# https://github.com/kupieckozienice/pyPDF2

# !git clone -l -s repo_link cloned-repo
# %cd cloned-repo
# !ls

# `instalowanie bibliotek`

In [173]:
# @title install unidecode { form-width: "500px", display-mode: "both" }
%pip install unidecode

In [174]:
# @title install python-docx { form-width: "750px", display-mode: "both" }
%pip install python-docx

In [175]:
# @title install PyPDF2 { form-width: "600px", display-mode: "both" }
%pip install PyPDF2

In [176]:
# @title install pdfminer.six { form-width: "600px", display-mode: "both" }
%pip install pdfminer.six

In [177]:
# @title install openpyxl { form-width: "600px", display-mode: "both" }
%pip install openpyxl

# `import dependencies`

In [178]:
# @title import dependencies { vertical-output: true, form-width: "500px", display-mode: "both" }
# @markdown instalowanie zależności:
# @markdown  `os`,
# @markdown  `pathlib`, `typing`

import os
import pathlib as pth
from typing import Union, Literal, List
from PyPDF2 import PdfReader, PdfWriter
from pdfminer.high_level import extract_text as fallback_text_extraction
from google.colab import files
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from dateutil import tz
import openpyxl
from openpyxl.reader.excel import load_workbook
import pandas as pd
from openpyxl.styles import Alignment, Font
import re
from unidecode import unidecode
import random
import string
import time
import random
from datetime import datetime, timedelta

# `operacje systemowe`

In [179]:
# @title rozdziel nazwe pliku od rozszerzenia { vertical-output: true, form-width: "500px", display-mode: "both" }
filename = "" # @param {type:"string"}

def split_filename_and_extension(filename):
  nazwa_pliku, rozszerzenie_pliku = os.path.splitext(filename)

  return {'name_key': nazwa_pliku, 'ext_key': rozszerzenie_pliku}

In [180]:
# @title zmień katalog domyślny na /content { vertical-output: true, form-width: "500px", display-mode: "both" }
new_directory_name = "/content" # @param {type:"string"}
os.chdir(new_directory_name)
print(os.getcwd())

/content


In [181]:
# @title konwertuje format daty { vertical-output: true, form-width: "500px", display-mode: "both" }
pd = "" # @param {type:"string"}

def convertPdfDatetime(pd):
    dtformat = "%Y%m%d%H%M%S"
    clean = pd.replace("D:","").split('+')[0]
    return datetime.strptime(clean,dtformat)

In [182]:
# @title iteruj pliki w podanym katalogu { vertical-output: true, form-width: "500px", display-mode: "both" }

directory_path = "" # @param {type:"string"}

def iterate_over_files_in_directory(path_to_directory):

  numerate = 1

  for filename in os.listdir(path_to_directory):
    f = os.path.join(path_to_directory, filename)
    if os.path.isfile(f):
      print(f"{numerate}. {f}")
      numerate = numerate + 1

In [268]:
# @title sprawdzenie czy istnieje katalog roboczy, w którym chcemy pracować { vertical-output: true, form-width: "500px", display-mode: "both" }
directory_path = "" # @param {type:"string"}

def sprawdz_czy_katalog_istnieje(directory_name):
  # Ścieżka do katalogu bieżącego
  cwd = os.getcwd()
  print(f"katalog \"bieżący\": {cwd}")

  working_directory_for_creating = os.path.join(cwd, directory_name)
  print(f"katalog roboczy do utworzenia (jeśli nie istnieje): {working_directory_for_creating}")

  try:
    if not os.path.exists(working_directory_for_creating):
      print(f"katalog: {working_directory_for_creating} nie istnieje i zostanie utworzony")
      os.makedirs(working_directory_for_creating)
    else:
      print(f"katalog: {working_directory_for_creating} już istnieje!")
  except FileExistsError:
    print(f"katalog o nazwie {working_directory_for_creating} już istnieje!")
  except Exception as e:
    print(f"wystąpił błąd podczas tworzenie katalogu: {working_directory_for_creating}")

# testy
sprawdz_czy_katalog_istnieje("/content/zz")

katalog "bieżący": /content
katalog roboczy do utworzenia (jeśli nie istnieje): /content/zz
katalog: /content/zz nie istnieje i zostanie utworzony


In [184]:
# @title importuj pliki do wskazanego katalogu { vertical-output: true, form-width: "500px", display-mode: "both" }
destination_directory_path = "" # @param {type:"string"}

def import_files(destination_directory_path):
  uploaded = files.upload()
  print(uploaded.keys())

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  for filename, file_content in uploaded.items():
    if os.path.exists(destination_directory_path) and os.path.isdir(destination_directory_path):
      with open(os.path.join(destination_directory_path, filename), "wb") as f:
        f.write(file_content)

      # remove files from /content
      file_to_remove_after_copying_to_working_directory = os.path.join(os.getcwd(), filename)
      if os.path.isfile(file_to_remove_after_copying_to_working_directory):
        os.remove(file_to_remove_after_copying_to_working_directory)
        if not os.path.exists(file_to_remove_after_copying_to_working_directory):
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")
        else:
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"NIE usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")

In [414]:
# @title sprawdź czy podany plik istnieje { vertical-output: true, form-width: "500px", display-mode: "both" }

path_to_file = "" # @param {type:"string"}
# @markdown sprawdza czy plik o podanej ścieżce istnieje

def sprawdz_czy_dany_plik_istnieje(path_to_file):
  try:
    with open(path_to_file, 'r') as file:
      print(f"Plik {path_to_file} istnieje i ma się całkiem nieźle ;-)")
      return True
  except FileNotFoundError:
    print('Sorry the file we\'re looking for doesn\' exist')
    return False
  except Exception as e:
    print(f"Wystąpił błąd: {e}")
    return False

In [186]:
# @title upload files { vertical-output: true, form-width: "500px", display-mode: "both" }
folder_domyslny = "" # @param {type:"string"}

def upload_files(folder_domyslny = "uploaded"):
  check_if_directory_and_path_exists(folder_domyslny)
  import_files(folder_domyslny)

In [187]:
# @title rozdziel nazwe pliku od rozszerzenia { vertical-output: true, form-width: "500px", display-mode: "both" }
filename = "" # @param {type:"string"}

def split_filename_and_extension(filename):
  nazwa_pliku, rozszerzenie_pliku = os.path.splitext(filename)

  return {'name_key': nazwa_pliku, 'ext_key': rozszerzenie_pliku}

In [188]:
# @title aktualna data i godzina { vertical-output: true, form-width: "500px", display-mode: "both" }

def actual_datetime():
  teraz = datetime.now(tz=tz.gettz('Europe/Warsaw'))
  formatted_date = teraz.strftime("%d_%m_%Y")
  formatted_time = teraz.strftime("%H_%M_%S")

  # print(formatted_date)
  # print(formatted_time)

  return {'data': formatted_date, 'czas': formatted_time}


In [258]:
# @title zamienia wszystkie inne znaki niż litery i cyfry (oprócz ost. kropki) na '_' { form-width: "600px", display-mode: "both" }
nazwa_pliku_do_zmiany_znakow = "" # @param {type:"string"}

# KZP-ŁA.2110.01.2024 Dostawa siłownika Auma


def zamien_znaki_w_nazwie_pliku(nazwa_pliku_do_zmiany_znakow):
  nazwa, rozszerzenie = nazwa_pliku_do_zmiany_znakow.rsplit('.',1) if '.' in nazwa_pliku_do_zmiany_znakow else (nazwa_pliku_do_zmiany_znakow, '')
  nazwa_bez_polskich_znakow = unidecode(nazwa)
  zmodyfikowana_nazwa = re.sub(r'[^a-zA-Z0-9]', '_', nazwa_bez_polskich_znakow)
  zmodyfikowany_string = zmodyfikowana_nazwa + ('.' + rozszerzenie if rozszerzenie else '')
  return zmodyfikowany_string

# testy
input_string = "1.Załącznik Zakres Przedmiotu zamówienia KZP-ŁA.2110.01.2024 Dostawa siłownika Auma — kopia.xlsx"
result = zamien_znaki_w_nazwie_pliku(input_string)
print(result)


1_Zalacznik_Zakres_Przedmiotu_zamowienia_KZP_LA_2110_01_2024_Dostawa_silownika_Auma____kopia.xlsx


In [189]:
# @title iteruje przez podany katalog i wyświetle nazwy plików { vertical-output: true, form-width: "500px", display-mode: "both" }
path_to_directory = "" # @param {type:"string"}

def iterate_over_files_in_directory(path_to_directory):

  numerate = 1

  for filename in os.listdir(path_to_directory):
    f = os.path.join(path_to_directory, filename)
    if os.path.isfile(f):
      print(f"{numerate}. {f}")
      numerate = numerate + 1

In [190]:
# @title sprawdzenie czy istnieje katalog roboczy, w którym chcemy pracować { vertical-output: true, form-width: "500px", display-mode: "both" }
directory_name_path = "" # @param {type:"string"}

def check_if_directory_and_path_exists(directory_name):
  # Ścieżka do katalogu bieżącego
  cwd = os.getcwd()
  print(f"katalog \"bieżący\": {cwd}")

  working_directory_for_creating = os.path.join(cwd, directory_name)
  print(f"katalog roboczy do utworzenia (jeśli nie istnieje): {working_directory_for_creating}")

  try:
    if not os.path.exists(working_directory_for_creating):
      print(f"katalog: {working_directory_for_creating} nie istnieje i zostanie utworzony")
      os.makedirs(working_directory_for_creating)
    else:
      print(f"katalog: {working_directory_for_creating} już istnieje!")
  except FileExistsError:
    print(f"katalog o nazwie {working_directory_for_creating} już istnieje!")
  except Exception as e:
    print(f"wystąpił błąd podczas tworzenie katalogu: {working_directory_for_creating}")

In [191]:
# @title importuj pliki do wskazanego katalogu { vertical-output: true, form-width: "500px", display-mode: "both" }
destination_directory = "" # @param {type:"string"}

def import_files(destination_directory):

  uploaded = files.upload()
  # print(uploaded.keys())

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  for filename, file_content in uploaded.items():
    if os.path.exists(destination_directory) and os.path.isdir(destination_directory):
      with open(os.path.join(destination_directory, filename), "wb") as f:
        f.write(file_content)

      # remove files from /content
      file_to_remove_after_copying_to_working_directory = os.path.join(os.getcwd(), filename)
      if os.path.isfile(file_to_remove_after_copying_to_working_directory):
        os.remove(file_to_remove_after_copying_to_working_directory)
        if not os.path.exists(file_to_remove_after_copying_to_working_directory):
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")
        else:
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"NIE usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")

In [192]:
# @title check if file exists { vertical-output: true, form-width: "500px", display-mode: "both" }
path_to_file = "" # @param {type:"string"}

def check_if_file_exists(path_to_file):
  try:
    with open(path_to_file, 'r') as file:
      print(f"Plik {path_to_file} istnieje i ma się całkiem nieźle ;-)")
      return True
  except FileNotFoundError:
    print('Sorry the file we\'re looking for doesn\' exist')
    return False
  except Exception as e:
    print(f"Wystąpił błąd: {e}")
    return False

In [193]:
# @title Generator Nazw Plikow { vertical-output: true, form-width: "500px", display-mode: "both" }
class GeneratorNazwPlikow:
  static_variable = 0

  def generuj_nazwe_pliku(self, nazwa_pliku):
    GeneratorNazwPlikow.static_variable += 1
    file_name = f"{nazwa_pliku}_{GeneratorNazwPlikow.static_variable}.docx"

    return file_name

In [194]:
# @title czyszczenie zawartości pliku { vertical-output: true, form-width: "500px", display-mode: "both" }

def wyczysc_zawartosc_pliku(sciezka_do_pliku):
  with open(sciezka_do_pliku, "w") as plik:
    plik.truncate(0)

In [195]:
# @title utwórz nowy plik { vertical-output: true, form-width: "500px", display-mode: "both" }

def utworz_nowy_plik(nazwa_pliku):
  aktualna_data_i_godzina = datetime.now().strftime("%d.%m.%Y %H.%M.%S %Z")
  print("aktualna_data_i_godzina:", aktualna_data_i_godzina)
  sciezka_do_pliku = f'/content/{nazwa_pliku}'

  with open(sciezka_do_pliku, "w") as plik:
    plik.write(aktualna_data_i_godzina)


# `Excel`

In [230]:
# @title kod w języku Python, który generuje losowe liczby w formacie "XXX-XXX-XXX-X" { vertical-output: true, form-width: "500px", display-mode: "both" }

def generuj_losowy_numer():
    numer = ""
    for _ in range(3):
        numer += str(random.randint(100, 999)) + "-"
    numer += str(random.randint(0, 9))
    return numer

  # Przykład użycia
losowy_numer = generuj_losowy_numer()
print(losowy_numer)

272-162-659-5


In [231]:
# @title kod w języku Python, który generuje losową nazwę w formacie "przedmiot nr [losowy tekst 10 literowy] [aktualny czas w milisekundach]" { vertical-output: true, form-width: "500px", display-mode: "both" }

def generuj_losowy_tekst(dlugosc):
    litery = string.ascii_letters
    return ''.join(random.choice(litery) for _ in range(dlugosc))

def generuj_losowa_nazwe():
    losowy_tekst = generuj_losowy_tekst(10)
    czas_w_milisekundach = int(round(time.time() * 1000))
    nazwa = f"przedmiot nr {losowy_tekst} {czas_w_milisekundach}"
    return nazwa

# Przykład użycia
losowa_nazwa = generuj_losowa_nazwe()
print(losowa_nazwa)


przedmiot nr mEbwkuNHpo 1705579949585


In [234]:
# @title kod w języku Python, który generuje losową liczbę całkowitą z podanego zakresu { vertical-output: true, form-width: "500px", display-mode: "both" }

variable_name_1 = 1 # @param {type:"integer"}
variable_name_2 = 100 # @param {type:"integer"}

def generuj_losowa_liczbe(variable_name_1, variable_name_2):
    return random.randint(variable_name_1, variable_name_2)

# Przykład użycia
zakres_od = 1
zakres_do = 100
losowa_liczba = generuj_losowa_liczbe(variable_name_1, variable_name_2)
print(losowa_liczba)



77


In [243]:
# @title kod w języku Python, który generuje losową datę z podanego przedziału i formatuje ją w formacie "dd.mm.rrrr" { vertical-output: true, form-width: "500px", display-mode: "both" }
zakres_poczatek_str = "2024-01-16" # @param {type:"date"}
zakres_koniec_str = "2024-02-08" # @param {type:"date"}

# Konwersja na obiekty datetime
zakres_poczatek = datetime.strptime(zakres_poczatek_str, "%Y-%m-%d")
zakres_koniec = datetime.strptime(zakres_koniec_str, "%Y-%m-%d")

def generuj_date_plus_dwa_tygodnie():
    aktualna_data = datetime.now()
    data_plus_dwa_tygodnie = aktualna_data + timedelta(days=14)
    return data_plus_dwa_tygodnie.strftime("%d.%m.%Y")


def generuj_losowa_date(zakres_poczatek, zakres_koniec):
    zakres_dat = zakres_koniec - zakres_poczatek
    print("zakres_dat: \t\t", zakres_dat)
    losowa_liczba_dni = random.randint(0, zakres_dat.days)
    losowa_data = zakres_poczatek + timedelta(days=losowa_liczba_dni)
    return losowa_data.strftime("%d.%m.%Y")

# Przykład użycia
zakres_poczatek = datetime(2024, 1, 1)
print("zakres_poczatek:\t", zakres_poczatek)
zakres_koniec = datetime(2024, 12, 31)
print("zakres_koniec:\t\t", zakres_koniec)

losowa_data = generuj_losowa_date(zakres_poczatek, zakres_koniec)
print("losowa_data: \t\t", losowa_data)

zakres_poczatek:	 2024-01-01 00:00:00
zakres_koniec:		 2024-12-31 00:00:00
zakres_dat: 		 365 days, 0:00:00
losowa_data: 		 18.02.2024


In [246]:
# @title generuje losową liczbę zmiennoprzecinkową (float) w podanym zakresie { vertical-output: true, form-width: "500px", display-mode: "both" }
zakres_od = 10 # @param {type:"number"}
zakres_do = 3455 # @param {type:"number"}

def generuj_losowa_liczbe_float(zakres_od, zakres_do):
    losowa_liczba_float = random.uniform(zakres_od, zakres_do)
    return round(losowa_liczba_float, 2)

# Przykład użycia
zakres_od = 1.0
zakres_do = 10.0
losowa_liczba_float = generuj_losowa_liczbe_float(zakres_od, zakres_do)
print(losowa_liczba_float)

4.89


In [257]:
# @title generuje losową liczbę całkowitą i zmiennoprzecinkową przy użyciu wcześniej zdefiniowanych funkcji generuj_losowa_liczbe i generuj_losowa_liczbe_float, a następnie mnoży te liczby { vertical-output: true, form-width: "600px", display-mode: "both" }

def generuj_losowa_liczbe():
    return random.randint(1, 100)

def generuj_losowa_liczbe_float():
    return round(random.uniform(1.0, 250.0), 2)

# Wygeneruj losowe liczby
liczba_calkowita = generuj_losowa_liczbe()
liczba_zmiennoprzecinkowa = generuj_losowa_liczbe_float()

# Pomnóż liczby
wynik_mnozenia = round(liczba_calkowita * liczba_zmiennoprzecinkowa, 2)

# Wyświetl wyniki
print(f"Liczba całkowita: {liczba_calkowita}")
print(f"Liczba zmiennoprzecinkowa: {liczba_zmiennoprzecinkowa}")
print(f"Wynik mnożenia zaokrąglony do 2 miejsc po przecinku: {wynik_mnozenia}")

Liczba całkowita: 24
Liczba zmiennoprzecinkowa: 41.22
Wynik mnożenia zaokrąglony do 2 miejsc po przecinku: 989.28


In [250]:
# @title generuj nazwe pliku { vertical-output: true, form-width: "500px", display-mode: "both" }

def generuj_nazwe_pliku():
    numer = random.randint(10000, 99999)
    nazwa_pliku = f"Zapotrzebzakupu1{numer}.xlsx"
    return nazwa_pliku

# Przykład użycia
nazwa_pliku = generuj_nazwe_pliku()
print(nazwa_pliku)

Zapotrzebzakupu177165.xlsx


In [253]:
# @title sparsuj nazwę pliku { vertical-output: true, form-width: "500px", display-mode: "both" }
nazwa_pliku = "" # @param {type:"string"}
nowe_rozszerzenie = "" # @param {type:"string"}


# nazwa_pliku = "Zapotrzebuzakupu123921.xlsx"

def sparsuj_i_utworz_nazwe(nazwa_pliku, nowe_rozszerzenie):
    wynik = re.search(r'\d+', nazwa_pliku)

    if wynik:
        sparsowana_liczba = wynik.group()
        nowa_nazwa_pliku = f"{sparsowana_liczba}.{nowe_rozszerzenie}"
        return nowa_nazwa_pliku
    else:
        print("Nie znaleziono liczby w nazwie pliku.")
        return None

# Przykład użycia
nazwa_pliku = "Zapotrzebuzakupu123921.xlsx"
nowe_rozszerzenie = "xlsx"
nowa_nazwa = sparsuj_i_utworz_nazwe(nazwa_pliku, nowe_rozszerzenie)

if nowa_nazwa:
    print(nowa_nazwa)

123921.xlsx


In [277]:
# @title utworz plik excel (szablon z nagłówkami itp) { vertical-output: true, form-width: "500px", display-mode: "both" }

def utworz_plik_excel(znak_sprawy):
    wb = openpyxl.Workbook()
    arkusz = wb.active

    # Nagłówki
    naglowki = ['lp', 'Indeks', 'Opis pozycji', 'Ilość', 'Jednostka masy', 'Data dostawy', 'Cena jednostkowa netto',
                'Wartość kontraktu netto', 'Osoba do kontaktu']

    # Wklejanie nagłówków w wierszu 3, zaczynając od kolumny 1
    for index, naglowek in enumerate(naglowki, start=1):
        komorka = arkusz.cell(row=3, column=index, value=naglowek)
        komorka.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
        komorka.font = Font(bold=True)

    # Wiersz 1
    komorka_wiersz_1 = arkusz.cell(row=1, column=1, value=f"Zapytanie ofertowe RFX nr sprawy {znak_sprawy}")
    komorka_wiersz_1.alignment = Alignment(horizontal='left', vertical='center', wrap_text=False)
    komorka_wiersz_1.font = Font(size=20, bold=True)

    # Wiersz 2
    data_wygenerowania = datetime.now().strftime("%Y-%m-%d")
    komorka_wiersz_2 = arkusz.cell(row=2, column=1, value=f"Data wygenerowania dokumentu: {data_wygenerowania}")
    komorka_wiersz_2.alignment = Alignment(horizontal='left', vertical='center', wrap_text=False)

    nazwa_pliku = f"{znak_sprawy}.xlsx"
    wb.save(nazwa_pliku)
    files.download(nazwa_pliku)

    print(f"Plik Excela '{nazwa_pliku}' został utworzony pomyślnie.")

    return nazwa_pliku


In [206]:
# @title przetwarzaj pliki excel { vertical-output: true, form-width: "500px", display-mode: "both" }
directory_path = "" # @param {type:"string"}

def przetwarzaj_pliki_excel(sciezka_katalogu):
    # Zamień wszystkie ukośniki na /
    sciezka_katalogu = sciezka_katalogu.replace('\\', '/')

    # Upewnij się, że ścieżka ma poprawny separator na końcu
    if not sciezka_katalogu.endswith('/'):
        sciezka_katalogu += '/'

    # Inicjalizuj kolekcje kolumn
    kolumny_do_skopiowania = ['B', 'C', 'D', 'E', 'F', 'K', 'S']
    wyniki = []

    # Pętla po plikach Excel w podanym katalogu
    for plik in os.listdir(sciezka_katalogu):
        if plik.lower().endswith((".xlsx", ".xls")):  # Sprawdź, czy plik to Excel (ignoruj wielkość liter)
            pelna_sciezka = os.path.join(sciezka_katalogu, plik)
            # print(pelna_sciezka)

            # Sprawdź, czy ścieżka do pliku istnieje, a jeśli nie, spróbuj dokonać zamiany ukośników
            if not os.path.exists(pelna_sciezka):
                pelna_sciezka = pelna_sciezka.replace('/', '\\') if '/' in pelna_sciezka else pelna_sciezka.replace(
                    '\\', '/')

            # Wczytaj arkusz z pliku Excel
            df = pd.read_excel(pelna_sciezka, engine='openpyxl')  # Użyj openpyxl jako silnika do wczytywania Excel
            print(df)

            # Sprawdź, czy wszystkie kolumny istnieją w ramce danych
            if set(kolumny_do_skopiowania).issubset(df.columns):
                # Skopiuj wybrane kolumny
                skopiowane_dane = df[kolumny_do_skopiowania].copy()
                print(skopiowane_dane)

                # Dodaj do wynikowej kolekcji
                wyniki.append(skopiowane_dane)
            else:
                print(f'Plik {plik} nie zawiera wszystkich kolumn.')

    # Zwróć złączone wyniki
    if wyniki:
        return pd.concat(wyniki, axis=0, ignore_index=True)
    else:
        return None




In [433]:
# @title utwórz baze danych osób do kontaktu { vertical-output: true, form-width: "600px", display-mode: "both" }

def singleton(cls):
  instances = {}

  def get_instance(*args, **kwargs):
    if cls not in instances:
      instances[cls] = cls(*args, **kwargs)
    return instances[cls]

  return get_instance

@singleton
class Utworz_baze_kontaktow:

  def __init__(self):
    # zawiera liste slownikow kontaktow imie nazwisko : reszta danych
    self._baza = []

  @property
  def get_baza(self):
    return self._baza

  def utworz_kontakt(self, linia_kontaktu):
    parts = linia_kontaktu.strip().split(',')
    key = parts[0]
    value = '.'.join(parts[1:]).strip()
    self.get_baza.append({key : value})

  def otworz_plik_z_kontaktami(self, kodowanie = "windows-1250"):
    sciezka_do_pliku_kontaktow = "/content/uploaded/ludzie_do_kontaktu_po_RFX — kopia.txt"
    if not sprawdz_czy_dany_plik_istnieje(sciezka_do_pliku_kontaktow):
      upload_files()

    with open(sciezka_do_pliku_kontaktow, 'r', encoding=kodowanie) as file:
      for linia_kontaktu in file:
        self.utworz_kontakt(linia_kontaktu)

  def wyswietl_baze_kontaktow(self):
    for item in self.get_baza:
      for key, value in item.items():
        print(f"{key}, {value}")

  def zapisz_baze_do_pliku(self, sciezka, download = False):
    with open(sciezka, "w") as plik_bazy:
      for rekord in self.get_baza:
        for key, value in rekord.items():
          plik_bazy.write(key + ", " + value + "\n")

    if(download): files.download(sciezka)


# test

utworz_baze_kontaktow = Utworz_baze_kontaktow()
utworz_baze_kontaktow.otworz_plik_z_kontaktami()
utworz_baze_kontaktow.wyswietl_baze_kontaktow()
baza = utworz_baze_kontaktow.get_baza

# for rekord in baza:
#   for key, val in rekord.items():
#     print(key, val)

utworz_baze_kontaktow.zapisz_baze_do_pliku("baza.txt")

Plik /content/uploaded/ludzie_do_kontaktu_po_RFX — kopia.txt istnieje i ma się całkiem nieźle ;-)
Grzegorz Grabski, tel. +48 48 614 1785. +48 605 902 570. e-mail: grzegorz.grabski@enea.pl
Hubert Chmielewski, tel. +48 607 045 281.486141943. e-mail: hubert.chmielewski@enea.pl
Dominik Ozimek, tel. +48 48 614 1495. e-mail: dominik.ozimek@enea.pl
Anna Smargol-Bociek, tel. +48 48 614 1689. e-mail: anna.smargol@enea.pl
Daniel Kęska, tel. +48 48 614 1637. e-mail: daniel.keska@enea.pl
Krzysztof Bitner, tel. +48 48 614 1936. e-mail: krzysztof.bitner@enea.pl TAZ
Izabela Chołuj, tel. +48 48 614 1942. e-mail: izabela.choluj@enea.pl
Krzysztof Jastrzębski, tel. +48 48 614 1922. e-mail: krzysztof.jastrzebski@enea.pl
Dariusz Chołuj, tel. +48486141921. +48695901220. e-mail: dariusz.choluj@enea.pl
Monika Petrykowska, tel. +48486141905. e-mail: monika.petrykowska@enea.pl
Norbert Kisiel, tel. +48486141671. +48691450023. e-mail: norbert.kisiel@enea.pl TWE
Sławomir Kapciak, tel. +48486141911. +48605470461. e

In [207]:
# @title skopiuj kolumny z katalogu { vertical-output: true, form-width: "500px", display-mode: "both" }

directory_path = "" # @param {type:"string"}

def skopiuj_kolumny_z_katalogu(sciezka_katalogu):
    try:
        lista_wynikowa = []

        # Sprawdź, czy ścieżka katalogu ma poprawny separator na końcu
        if not sciezka_katalogu.endswith(os.path.sep):
            sciezka_katalogu += os.path.sep

        # Indeksy kolumn do skopiowania
        kolumny_do_skopiowania = [2, 3, 4, 5, 6, 11, 19]  # Kolumny B, C, D, E, F, K, S

        # Pętla po plikach Excel w podanym katalogu
        for plik in os.listdir(sciezka_katalogu):
            if plik.lower().endswith((".xlsx", ".xls")):  # Sprawdź, czy plik to Excel (ignoruj wielkość liter)
                pelna_sciezka = os.path.join(sciezka_katalogu, plik)

                # Wczytaj plik Excel
                workbook = openpyxl.load_workbook(pelna_sciezka)
                sheet = workbook.active

                # Inicjalizuj listę krotek dla bieżącego pliku
                lista_krotek = []

                # Iteruj po kolumnach i dodawaj każdą do osobnej krotki
                for kolumna_index in kolumny_do_skopiowania:
                    krotka = tuple(
                        sheet.komórka(row=row, column=kolumna_index).value for row in range(2, sheet.max_row + 1))
                    lista_krotek.append(krotka)

                # Dodaj listę krotek dla bieżącego pliku do listy wynikowej
                lista_wynikowa.append(lista_krotek)

        return lista_wynikowa
    except Exception as e:
        print(f"Błąd podczas przetwarzania plików Excel: {e}")
        return None


In [208]:
# @title dodaj krotki { vertical-output: true, form-width: "500px", display-mode: "both" }
lista_wynikowa = "" # @param {type:"string"}

def dodaj_krotki(lista_wynikowa):
    if lista_wynikowa is not None:
        suma_krotek = []

        # Iteruj po indeksach krotek (kolumnach)
        for indeks_kolumny in range(len(lista_wynikowa[0])):
            suma_kolumny = ()

            # Iteruj po plikach i dodawaj odpowiadające sobie krotki
            for lista_krotek in lista_wynikowa:
                suma_kolumny += lista_krotek[indeks_kolumny]

            suma_krotek.append(suma_kolumny)

        return suma_krotek
    else:
        print("Błąd podczas przetwarzania listy krotek.")
        return None


In [209]:
# @title oblicz najdluzszy string w kolumnie { vertical-output: true, form-width: "500px", display-mode: "both" }

kolumna = "" # @param {type:"string"}

def oblicz_najdluzszy_string(kolumna):
    return max(len(str(wartosc)) for wartosc in kolumna if isinstance(wartosc, str))


In [210]:
# @title zapisz sumy do excel { vertical-output: true, form-width: "500px", display-mode: "both" }

def zapisz_sumy_do_excel(suma_krotek, sciezka_do_pliku_excel):
    try:
        # Utwórz nowy plik Excel
        wb = load_workbook(sciezka_do_pliku_excel)

        # nowy_workbook = openpyxl.Workbook()
        nowy_sheet = wb.active

        # Ustaw szerokość kolumn do zawartości
        for kolumna_index, suma_kolumny in enumerate(suma_krotek, start=2):
            szerokosc_kolumny = max(len(str(wartosc)) for wartosc in suma_kolumny)
            nowy_sheet.column_dimensions[openpyxl.utils.get_column_letter(kolumna_index)].width = szerokosc_kolumny

        # Ustawienia formatowania
        align_center = Alignment(horizontal='center', vertical='center', wrap_text=True)
        date_format = '%Y.%m.%d'

        # Zapisz sumy kolumn do kolumny B, zaczynając od wiersza 4
        for indeks, suma_kolumny in enumerate(suma_krotek, start=1):
            for nr_wiersza, wartosc in enumerate(suma_kolumny, start=4):
                komorka = nowy_sheet.komórka(row=nr_wiersza, column=indeks + 1, value=wartosc)

                # Wysrodkuj komórkę
                komorka.alignment = align_center

                # Ustaw format daty w kolumnie C
                if indeks == 4:
                    if isinstance(wartosc, datetime):
                        komorka.number_format = date_format

        # Zapisz plik Excel
        wb.save(sciezka_do_pliku_excel)
        print(f"Sumy kolumn zostały zapisane do pliku Excel: {sciezka_do_pliku_excel}")
    except Exception as e:
        print(f"Błąd podczas zapisywania pliku Excel: {e}")


In [269]:
# @title praca z plikiem excel { vertical-output: true, form-width: "500px", display-mode: "both" }

def praca_z_plikiem_excel():

  # utworzenie katalogu do pracy z biblioteka openpyxl

  try:
    dir_name_for_openpyxl = "openpyxl"
    sprawdz_czy_katalog_istnieje(dir_name_for_openpyxl)

    wb = Workbook()
    ws = wb.active

    data = [
        ['Apples', 10000, 5000, 8000, 6000],
        ['Pears',   2000, 3000, 4000, 5000],
        ['Bananas', 6000, 6000, 6500, 6000],
        ['Oranges',  500,  300,  200,  700],
    ]

    # add column headings. NB. these must be strings
    ws.append(["Fruit", "2011", "2012", "2013", "2014"])
    for row in data:
        ws.append(row)

    tab = Table(displayName="Table1", ref="A1:E5")

    # Add a default style with striped rows and banded columns
    style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                          showLastColumn=False, showRowStripes=True, showColumnStripes=True)
    tab.tableStyleInfo = style

    '''
    Table must be added using ws.add_table() method to avoid duplicate names.
    Using this method ensures table name is unque through out defined names and all other table name.
    '''
    ws.add_table(tab)

    # Save the file with actual date and time
    data_czas = actual_datetime()
    lista_daty = ["_", data_czas['czas'], "___", data_czas['data']]
    skrot_do_daty_i_godziny = "".join(lista_daty)

    podzielona_nazwa_pliku = split_filename_and_extension("sample.xlsx")
    nowa_nazwa_pliku_skladowe = [podzielona_nazwa_pliku.get('name_key'),
                                skrot_do_daty_i_godziny,
                                podzielona_nazwa_pliku.get('ext_key')]

    nowa_nazwa_pliku = "".join(nowa_nazwa_pliku_skladowe)

    wb.save(nowa_nazwa_pliku)

  except Exception as e:
    print(f"wyje.ało błęda: {e}")

In [434]:
# @title Excel MainAPP { form-width: "500px", display-mode: "both" }

utworz_plik_excel("KZP-ŁA.2110.03.2024 Dostawa skanerów płomienia Lamtec 1W")
# praca_z_plikiem_excel()
# przetwarzaj_pliki_excel("/content/zz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Plik Excela 'KZP-ŁA.2110.03.2024 Dostawa skanerów płomienia Lamtec 1W.xlsx' został utworzony pomyślnie.


'KZP-ŁA.2110.03.2024 Dostawa skanerów płomienia Lamtec 1W.xlsx'

# `word`

In [213]:
# @title klasa - encja punktów { vertical-output: true, form-width: "600px", display-mode: "both" }


class Punkt:
  levels_number = None

  def __init__(self, point_dict):
    self._levels_number = len(point_dict)
    self._dict_with_data = point_dict

  @property
  def get_levels_number(self):
    return self._levels_number

  @property
  def get_dict_with_data(self):
    return self._dict_with_data

In [214]:
# @title utwórz punkty { vertical-output: true, form-width: "600px", display-mode: "both" }
string_level_1 = "" # @param {type:"string"}
string_level_2 = "" # @param {type:"string"}
variable_name = ""


def utworz_punkty():
  level_1 = ["CENA OFERTOWA (CENA ZA WYKONANIE PRZEDMIOTU ZAMÓWIENIA):", "W ofercie Wykonawca poda Cenę ofertową w PLN w następującym układzie:"]
  level_2 = ["Cena ofertowa netto\t\t(wartościowo),", "Podatek VAT\t\t\t(wartościowo),", "Cena ofertowa brutto\t\t(wartościowo i słownie)."]
  levels = {"1": level_1,
            "2": level_2}

  # wyświetlenie parametrów punktu
  punkt_1 = Punkt(levels)
  # print("Liczba poziomów: ", punkt_1.get_levels_number)

  for key, value in punkt_1.get_dict_with_data.items():
    print(f"poziom: {key}")
    for vals in value:
      print(vals)

  doc = Document()

  if isinstance(levels, dict):
    for key, value in levels.items():
      if key == "1":
        paragraf = doc.add_paragraph(value, style = 'ListNumber')
      elif key == "2":
        for list_item in value:
          sub_paragraf = doc.add_paragraph(list_item, style = 'ListBullet')

  doc.add_page_break()

  doc.save("sample_word_levels.docx")
  files.download("sample_word_levels.docx")

In [215]:
# @title wczytaj paragrafy z pliku docs - analiza zawartości { form-width: "500px", display-mode: "both" }
nazwa_pliku = "" # @param {type:"string"}
upload_directory_path = "" # @param {type:"string"}

def wczytaj_paragrafy_z_pliku_word(nazwa_pliku, upload_directory_path):
    import_files(upload_directory_path)
    doc = Document(os.path.join(upload_directory_path, "demo.docx"))
    # for index, paragraf in enumerate(doc.paragraphs):
      # print(f"\n\n--> Paragraf nr {index} \n", paragraf.text)

# testy
# wczytaj_paragrafy_z_pliku_word("zz", "demo.docx")


In [216]:
# @title praca z plikiem word { vertical-output: true, form-width: "600px", display-mode: "both" }

def praca_z_plikiem_word():

  doc = Document()

  # modyfikacja stylu Normal
  normal_style = doc.styles['Normal']
  normal_style.font.name = 'Tahoma'
  normal_style.font.size = Pt(9)
  normal_style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.LEFT

  # styl nagłówka GŁÓWNEGO
  style = doc.styles.add_style('nagłówek_główny_styl', 1)
  style.font.name = 'Tahoma'
  style.font.size = Pt(9)
  style.paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER

  # zmiana marginesu storny
  sekcja = doc.sections[0]
  sekcja.left_margin = Inches(1)
  sekcja.right_margin = Inches(1)
  szerokosc_strony = sekcja.page_width - sekcja.left_margin - sekcja.right_margin

  # dodanie logo firmy
  logo_sciezka = '/content/uploaded/nagłowek_enea.PNG'

  if not check_if_file_exists(logo_sciezka):
    upload_files(os.path.dirname(logo_sciezka))
    logo_added = doc.add_picture(logo_sciezka)
    wciecie_logo = szerokosc_strony - logo_added.width / 2
  else:
    logo_added = doc.add_picture(logo_sciezka)
    wciecie_logo = szerokosc_strony - logo_added.width / 2

  # dane wejściowe
  nazwa_sprawy = "KZP.ŁA.2110.23.2024 Dostawa łożysk"
  data_dokumentu = datetime.now().strftime('%d.%m.%Y')
  # data_dokumentu = datetime.now().strftime('%d.%m.%Y %H:%M:%S')

  # dodanie znaku sprawy i daty
  p = doc.add_paragraph(f'Nasz znak: ')
  run_znak_sprawy = p.add_run(f'{nazwa_sprawy}')
  run_znak_sprawy.bold = True
  p.style = 'Normal'

  # dodanie daty dokumentu
  data = doc.add_paragraph(f"Świerże Górne, dn. ")
  data_run = data.add_run(f'{data_dokumentu}r.')
  data_run.bold = True
  data.alignment = WD_ALIGN_PARAGRAPH.RIGHT

  # dodanie nagłówka
  nagłówek_text = doc.add_heading('ZAPYTANIE OFERTOWE (RFX)', 0)
  nagłówek_text.style = 'nagłówek_główny_styl'

  pkt_1 = "WPROWADZENIE:\nNiniejsze Zapytanie jest podstawą do złożenia ofert."
  pkt_2 = "TRYB UDZIELENIA ZAMÓWIENIA:\nPostępowanie w trybie Zapytania Ofertowego (RFX) prowadzone jest w oparciu o wewnętrzny Regulamin Udzielania Zamówień i Zawierania Umów (RUZiZU) Zamawiającego, dla którego nie mają zastosowania przepisy ustawy z dnia 11.09.2019 r. Prawo zamówień publicznych."
  podpunkt_3_1 = f"Wykonanie usługi pn. \"{nazwa_sprawy}\"."
  podpunkt_3_2 = "Zakres dostawy wg: Załącznika Nr 1 pn. \"Zakres Przedmiotu zamówienia\"."
  pkt_3 = {"PRZEDMIOT ZAMÓWIENIA": [podpunkt_3_1, podpunkt_3_2]}
  pkt_4 = "OFERTY CZĘŚCIOWE:\nZamawiający dopuszcza składania ofert częściowych."
  pkt_5 = "OFERTY WARIANTOWE:\nZamawiający nie dopuszcza składania ofert wariantowych."
  podpunkt_6_1 = "rozpoczęcie wykonywania Przedmiotu umowy  – od dnia podpisania Zamówienia,"
  podpunkt_6_2 = f"zakończenie wykonywania Przedmiotu umowy – zgodnie z załącznikiem „1. Załącznik {nazwa_sprawy} — zakres przedmiotu zamówienia”; lub 1 tydzień od daty otrzymania zamówienia, termin ten uważany będzie za dochowany przez Wykonawcę w przypadku, gdy przed jego upływem nastąpi protokolarny, bezusterkowy odbiór końcowy prac będących Przedmiotem umowy,"
  pkt_6 = {"TERMINY REALIZACJI: ":[podpunkt_6_1, podpunkt_6_2]}
  podpunkt_7_1 = "Realizacja przedmiotu zamówienia polega na:  Dostawie dla Enea Wytwarzanie sp. z o.o. fabrycznie nowych, nieużywanych, nieregenerowanych wyrobów będących przedmiotem niniejszego zapytania ofertowego wykonanych zgodnie z opisem przedmiotu zamówienia."
  pkt_7 = {"SZCZEGÓŁOWE WARUNKI TECHNICZNE/TECHNOLOGICZNE REALIZACJI PRZEDMIOTU ZAMÓWIENIA":[podpunkt_7_1]}

  lista_punktów = [pkt_1, pkt_2, pkt_3, pkt_4, pkt_5, pkt_6, pkt_7]

  for punkt in lista_punktów:
    if isinstance(punkt, dict):
      for main_item, sub_items in punkt.items():
        paragraf = doc.add_paragraph(main_item, style = 'ListNumber')
        for sub_item in sub_items:
          sub_paragraf = doc.add_paragraph(sub_item, style = 'ListBullet')
          sub_paragraf.paragraph_format.left_indent = 360
    else:
      paragraf = doc.add_paragraph(punkt, style = 'ListNumber')

  doc.add_page_break()

  # złożenie nowej nazwy pliku wraz z nazwą sprawy
  podzielona_nazwa_pliku_slownik = split_filename_and_extension("Zapytanie_ofertowe_RFX.docx")
  nazwa_trzon = podzielona_nazwa_pliku_slownik['name_key']
  rozszerzenie = podzielona_nazwa_pliku_slownik['ext_key']

  skladowe_nazwy_pliku = [nazwa_trzon, "_", nazwa_sprawy, rozszerzenie]
  nowa_nazwa_pliku = "".join(skladowe_nazwy_pliku)
  print(nowa_nazwa_pliku)

  doc.save(nowa_nazwa_pliku)
  files.download(nowa_nazwa_pliku)

In [217]:
# @title main word app { form-width: "600px", display-mode: "both" }
# praca_z_plikiem_word()

# `PDF`

In [218]:
# @title convert Pdf date and time { vertical-output: true, form-width: "500px", display-mode: "both" }
pd = "" # @param {type:"string"}

def convert_PDF_datetime(pd):
    dtformat = "%Y%m%d%H%M%S"
    clean = pd.replace("D:","").split('+')[0]
    date_converted = datetime.strptime(clean,dtformat)
    print(date_converted)

    return datetime.strptime(clean,dtformat)

In [219]:
# @title dodaj pustą stronę do pliku PDF { vertical-output: true, form-width: "500px", display-mode: "both" }

writer = "" # @param {type:"string"}
path_destination = "" # @param {type:"string"}

def add_blank_page(writer, path_destination):
  writer.add_blank_page(width=200, height=200)

  data = b"any bytes - typically read from a file"
  writer.add_attachment("smile.png", data)

  with open("output.pdf", "wb") as output_stream:
      writer.write(output_stream)

In [220]:
# @title sprawdź czy plik PDF jest zaszyfrowany { vertical-output: true, form-width: "500px" }
path_to_file = "" # @param {type:"string"}

def check_if_encrypted(path_to_file):
  if os.path.isfile(path_to_file):
    reader = PdfReader(path_to_file)

    if reader.is_encrypted:
      writer = PdfWriter()
      password = input("Enter your password: ")
      reader.decrypt(password)

      # Add all pages to the writer
      for page in reader.pages:
          writer.add_page(page)

      # Save the new PDF to a file
      with open(path_to_file, "wb") as f:
          writer.write(f)

      print("plik rozszyfrowany")
    else:
      print("plik NIE był zaszyfrowany")

In [221]:
# @title czytaj adnotacje { vertical-output: true, form-width: "500px", display-mode: "both" }
reader = "" # @param {type:"string"}

def annots_reading(reader):
  print("funkcja annots_reading załadowana")

  attachments = {}

  for page in reader.pages:
    if "/Annots" in page:
        for annot in page["/Annots"]:
            obj = annot.get_object()
            annotation = {"subtype": obj["/Subtype"], "location": obj["/Rect"]}
            print(annotation)

            # text
            subtype = annot.get_object()["/Subtype"]
            if subtype == "/Text":
                print(annot.get_object()["/Contents"])

            # Highlights
            subtype = annot.get_object()["/Subtype"]
            if subtype == "/Highlight":
                coords = annot.get_object()["/QuadPoints"]
                x1, y1, x2, y2, x3, y3, x4, y4 = coords

            # # attachments
            # subtype = annot.get_object()["/Subtype"]
            # if subtype == "/FileAttachment":
            #     fileobj = annotobj["/FS"]
            #     attachments[fileobj["/F"]] = fileobj["/EF"]["/F"].get_data()
    else:
      # print("brak andotacji w pliku")
      pass

In [222]:
# @title Add the metadata { vertical-output: true, form-width: "500px", display-mode: "both" }
path_to_file = "" # @param {type:"string"}

def add_metadata(path_to_file):
  reader = PdfReader(path_to_file)
  writer = PdfWriter()

  # Add all pages to the writer
  for page in reader.pages:
      writer.add_page(page)

  writer.add_metadata(
      {
          "/Author": "Martino",
          "/Producer": "Libre Writer",
          "/Subject": "kikk",
          "/Title": "jakis_tytul",
          # "/ModDate": modification_date  # Adding modification date
      }
  )

  with open(os.path.join(path_to_working_directory, path_to_file), "wb") as f:
    writer.write(f)

In [223]:
# @title czytaj metadane { vertical-output: true, form-width: "500px", display-mode: "both" }

reader = "" # @param {type:"string"}

def read_metadata(reader):
  meta = reader.metadata
  print(len(reader.pages))

  # All of the following could be None!
  print(f"author: {meta.author}")
  print(f"creator: {meta.creator}")
  print(f"producer: {meta.producer}")
  print(f"subject: {meta.subject}")
  print(f"title: {meta.title}")

In [224]:
# @title reading form fields { vertical-output: true, form-width: "500px", display-mode: "both" }
reader = "" # @param {type:"string"}

def reading_form_fields(reader):
  # fields = reader.get_form_text_fields()
  # fields == {"key": "value", "key2": "value2"}
  # if reader.isEditable:
  fields = reader.get_fields()

  if bool(fields):
    # print(fields)
    for field_name, field_value in fields.items():
      print(f"nazwa pola: {field_name}, wartość: {field_value}")
  else:
    print("brak fields-ów")

  # for field_name in fields:
    # print(field_name)

In [225]:
# @title extract text { vertical-output: true, form-width: "500px", display-mode: "both" }
reader = "" # @param {type:"string"}

def extract_text(reader):
  text = ""
  try:
      for page in reader.pages:
          text += page.extract_text()
  except Exception as exc:
      text = fallback_text_extraction("example.pdf")

  return text

In [226]:
# @title usuwanie stron w pdf { vertical-output: true, form-width: "500px", display-mode: "both" }
source_file = "" # @param {type:"string"}
pages_nums_to_delete = 0 # @param {type:"integer"}

def page_deleting(source_file, pages_nums_to_delete):
  print(source_file)

  file_name_without_ext, ext = os.path.splitext(source_file)
  new_file_name = ''.join([os.path.basename(file_name_without_ext), ext])

  destination_file = os.path.join(os.path.dirname(source_file), new_file_name)
  print(file_name_without_ext)
  print(ext)
  print(new_file_name)

  with open(source_file, "rb") as file:
    reader = PdfReader(file)
    writer = PdfWriter()

    for page_num in range(len(reader.pages)):
      if page_num + 1 not in pages_nums_to_delete:
        page_to_stay = reader.pages[page_num]
        writer.add_page(page_to_stay)

  with open(destination_file, "wb") as new_file:
    writer.write(new_file)

  with open(destination_file, "rb") as new_file:
    print(f"liczba stron nowego pliku: {len(PdfReader(new_file).pages)}")

In [227]:
# @title PDF MainAPP { vertical-output: true, form-width: "500px", display-mode: "both" }

def pdf_operations():

  filename = os.path.join(os.getcwd(), "pypdf2", "oferta_handlowa_wzor.pdf")

  if check_if_file_exists(filename):
    pdf_operations()
  else:
    upload_files("pypdf2")
    pdf_operations()

  # usuwanie stron
  page_to_delete = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
  page_deleting(filename, page_to_delete)

  reader = PdfReader(filename)
  writer = PdfWriter()

  check_if_encrypted(filename)

  # annots reading
  annots_reading(reader)

  # czytanie metadanych
  meta = reader.metadata
  meta

  # iterate over files in directory
  # iterate_over_files_in_directory(path_to_working_directory)

  # Reading form fields
  reading_form_fields(reader)

  # extract text
  text = extract_text(reader)
  print(text)

  # add metadata
  # add_metadata(filename)

# Brudnopis

In [228]:
# @title Ćwiczenia z Word i python-docs { run: "auto", vertical-output: true, form-width: "600px", display-mode: "both" }

def praca_z_brudnopisem():
  generator_nazwy_pliku = GeneratorNazwPlikow()
  nazwa_pliku_wynikowego = generator_nazwy_pliku.generuj_nazwe_pliku("dokument")
  print("nazwa_pliku_wynikowego: ", nazwa_pliku_wynikowego)

  utworz_nowy_plik(nazwa_pliku_wynikowego)
  sciezka_do_pliku_word_do_wyczyszczenia = os.path.join("/content/", nazwa_pliku_wynikowego)
  wyczysc_zawartosc_pliku(sciezka_do_pliku_word_do_wyczyszczenia)

  moj_dokument = Document(sciezka_do_pliku_word_do_wyczyszczenia)
  paragraph = document.add_paragraph('Lorem ipsum dolor sit amet.')

  doc.save(nazwa_pliku_wynikowego)
  files.download(nazwa_pliku_wynikowego)

In [229]:
# praca_z_brudnopisem()

nazwa_pliku_wynikowego:  dokument_1.docx
aktualna_data_i_godzina: 18.01.2024 12.11.38 


PackageNotFoundError: Package not found at '/content/dokument_1.docx'